In [ ]:
# initiating gpu using tensorflow.
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import random
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import load_model

In [ ]:
train = '../input/train/'
test = '../input/test/'
img_size1 = 96
img_size2 = 96

In [ ]:
cancer_files_train = pd.read_csv('../input/train_labels.csv')

In [ ]:
cancer_files_train.head()

In [ ]:
for i in range(len(cancer_files_train)):
        img_array = cv2.imread(os.path.join(train,cancer_files_train['id'][i]+'.tif'),cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array, cmap='gray')
        plt.show()
        break

In [ ]:
# creating a training dataset.
training_data = []
i = 0
def create_training_data():
    for i in range(len(cancer_files_train)):
            img_array = cv2.imread(os.path.join(train,cancer_files_train['id'][i]+'.tif'),cv2.IMREAD_GRAYSCALE)
            new_img = cv2.resize(img_array,(img_size2,img_size1))
            training_data.append([
                new_img,cancer_files_train['label'][i]])

In [ ]:
# Creating a test dataset.
testing_data = []
i = 0
def create_testing_data():        
    for img in os.listdir(test):
        img_array = cv2.imread(os.path.join(test,img),cv2.IMREAD_GRAYSCALE)
        new_img = cv2.resize(img_array,(img_size2,img_size1))
        testing_data.append([img,
            new_img])

In [ ]:
create_training_data()

In [ ]:
create_testing_data()

In [ ]:
print(len(training_data))
print(len(testing_data))

In [ ]:
random.shuffle(training_data)

In [ ]:
x = []
y = []

for features, label in training_data:
    x.append(features)
    y.append(label)

X = np.array(x).reshape(-1,img_size2,img_size1,1)

In [ ]:
X.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=50)

Y_train = np_utils.to_categorical(y_train,num_classes=2)
Y_test = np_utils.to_categorical(y_test,num_classes=2)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32,kernel_size=(3,3),strides=2,activation='relu',input_shape=(img_size1,img_size1,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))

In [ ]:
model.add(Conv2D(64,kernel_size=(3,3),strides=2,activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))


In [ ]:
model.add(Conv2D(128,kernel_size=(3,3),strides=2,activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.5))

In [ ]:
model.add(Flatten())
model.add(Dense(units = 512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units = 128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
callbacks = [EarlyStopping(monitor='val_acc',patience=5)]

In [ ]:
batch_size = 50
n_epochs = 20
results = model.fit(x_train,Y_train,batch_size=batch_size,epochs=n_epochs,verbose=1,validation_data=(x_test,Y_test),callbacks=callbacks)

In [ ]:
model.save_weights('./driverdistraction_lr_weights.h5', overwrite=True)

In [ ]:
model.save('./driverdistraction_lr_weights.h5')

In [ ]:
model = load_model('../working/driverdistraction_lr_weights.h5')

In [ ]:
test_data = np.array(testing_data[0][1]).reshape(-1,img_size2,img_size1,1)

In [ ]:
preds = model.predict(test_data)